<h1>Table of Contents &lt;br&gt;&lt;/br&gt;<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Data-loading" data-toc-modified-id="Imports-and-Data-loading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Data loading</a></span></li></ul></div>

## Imports and Data loading

In [1]:
# Display plots inline
%matplotlib inline

# Autoreload all package before excecuting a call
%load_ext autoreload
%autoreload 2

In [2]:
import evsim
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


# ENV_NAME = 'CartPole-v0'
ENV_NAME = 'evsim-v0'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.

dqn.fit(env, nb_steps=6700, visualize=False, verbose=2)

# After training is done, we save the final weights.

dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.

dqn.test(env, nb_episodes=1, visualize=False)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


/home/morty/uni/evsim/env/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Could not seed environment <FleetEnv<evsim-v0>>
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                32        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

/home/morty/uni/evsim/env/lib/python3.7/site-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
[2017-02-23 23:30:01] - Controller(integrated) Retrieving clearing price failed: 2017-02-24 00:00:00 is not in data.
[2017-02-23 23:45:01] - Controller(integrated) Retrieving clearing price failed: 2017-02-24 00:15:00 is not in data.
[2017-02-24 00:30:01] - Controller(integrated) Retrieving clearing price failed: 2017-02-24 01:00:00 is not in data.
[2017-02-24 00:45:01] - Controller(integrated) Retrieving clearing price failed: 2017-02-24 01:15:00 is not in data.
[2017-02-24 01:00:01] - Controller(integrated) Retrieving clearing price failed: 2017-02-24 01:30:00 is not in data.
[2017-02-24 01:15:01] - Controller(integrated) Retrieving clearing price failed: 2017-02

...

[2017-04-28 23:00:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-05 23:00:00 is not in data.
[2017-04-28 23:15:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-05 23:15:00 is not in data.
[2017-04-28 23:30:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-05 23:30:00 is not in data.
[2017-04-28 23:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-05 23:45:00 is not in data.
[2017-04-29 00:00:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 00:00:00 is not in data.
[2017-04-29 00:15:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 00:15:00 is not in data.
[2017-04-29 00:30:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 00:30:00 is not in data.
[2017-04-29 00:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 00:45:00 

[2017-04-29 15:15:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 15:15:00 is not in data.
[2017-04-29 15:30:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 15:30:00 is not in data.
[2017-04-29 15:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 15:45:00 is not in data.
[2017-04-29 16:00:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 16:00:00 is not in data.
[2017-04-29 16:15:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 16:15:00 is not in data.
[2017-04-29 16:30:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 16:30:00 is not in data.
[2017-04-29 16:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 16:45:00 is not in data.
[2017-04-29 17:00:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-06 17:00:00 

[2017-04-30 07:30:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 07:30:00 is not in data.
[2017-04-30 07:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 07:45:00 is not in data.
[2017-04-30 08:00:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 08:00:00 is not in data.
[2017-04-30 08:15:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 08:15:00 is not in data.
[2017-04-30 08:30:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 08:30:00 is not in data.
[2017-04-30 08:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 08:45:00 is not in data.
[2017-04-30 09:00:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 09:00:00 is not in data.
[2017-04-30 09:15:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 09:15:00 

[2017-04-30 23:30:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-01 00:00:00 is not in data.
[2017-04-30 23:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-07 23:45:00 is not in data.
[2017-04-30 23:45:01] - Controller(integrated) Capacity prediction failed: 15 min timeslot 2017-05-01 00:15:00 is not in data.


Episode 1: reward: 16473.421, steps: 6429
